In [26]:
import pandas as pd
import os
import re

def combine_and_filter_csv(folder_path, column_name="合约代码"): #添加默认列名，防止出错
    """
    合并指定文件夹下的 CSV 文件，添加日期列，并删除指定列中包含“小计”和“合计”的行。

    Args:
        folder_path: 包含 CSV 文件的文件夹路径。
        column_name: 要检查的列名，默认为"Unnamed: 1"，如果你的csv文件的第二列有名字，请修改此参数。

    Returns:
        合并并过滤后的 pandas DataFrame。如果发生错误或没有找到 CSV 文件，则返回 None。
    """

    file_list = os.listdir(folder_path)
    csv_list = [f for f in file_list if f.endswith('.csv')]
    date_list = [f[:8] for f in csv_list]

    if not csv_list:
        print(f"文件夹 {folder_path} 中没有找到 CSV 文件。")
        return None

    all_df = []
    for filename, date in zip(csv_list, date_list):
        try:
            filepath = os.path.join(folder_path, filename)
            df = pd.read_csv(filepath, encoding='gbk')

            df.insert(0, '交易日期', date)
            all_df.append(df)
        except pd.errors.ParserError as e:
            print(f"解析文件 {filename} 时发生错误：{e}")
            return None
        except UnicodeDecodeError as e:
            print(f"文件 {filename} 解码错误：{e}，请检查编码")
            return None
        except FileNotFoundError as e:
            print(f"文件 {filename} 未找到：{e}")
            return None
        except Exception as e:
            print(f"处理文件 {filename} 时发生未知错误：{e}")
            return None
    if not all_df: #如果没有任何文件被成功读取，则返回None
        return None

    combined_df = pd.concat(all_df, ignore_index=True)

    try:
        # 使用正则表达式进行更灵活的匹配，忽略大小写和空格
        combined_df = combined_df[~combined_df[column_name].astype(str).str.contains(r"\s*(小计|合计)\s*", regex=True, case=False)]
    except KeyError as e:
        print(f"指定的列名 {column_name} 不存在，请检查CSV文件或column_name参数：{e}")
        return None
    except AttributeError as e:
        print(f"指定的列 {column_name} 可能不是字符串类型，请检查CSV文件：{e}")
        return None

    return combined_df

# 示例用法
folder_path = "csv_files"  # 替换为实际的文件夹路径
combined_df = combine_and_filter_csv('D:\JupyterLabFiles\Futures\Futures_exchange_data\CFFEX') # 使用默认列名"Unnamed: 1"

if combined_df is not None:
    print(combined_df.head())
    combined_df.to_csv("combined_filtered.csv", index=False, encoding='gbk')
else:
    print("处理过程中出现错误。")

<>:64: SyntaxWarning: invalid escape sequence '\J'
<>:64: SyntaxWarning: invalid escape sequence '\J'
C:\Users\qssch\AppData\Local\Temp\ipykernel_11252\1941040054.py:64: SyntaxWarning: invalid escape sequence '\J'
  combined_df = combine_and_filter_csv('D:\JupyterLabFiles\Futures\Futures_exchange_data\CFFEX') # 使用默认列名"Unnamed: 1"
C:\Users\qssch\AppData\Local\Temp\ipykernel_11252\1941040054.py:52: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  combined_df = combined_df[~combined_df[column_name].astype(str).str.contains(r"\s*(小计|合计)\s*", regex=True, case=False)]


       交易日期                            合约代码     今开盘     最高价     最低价     成交量  \
0  20100416  IF1005                          3450.0  3488.0  3413.2   48988   
1  20100416  IF1006                          3470.0  3517.6  3440.0    4908   
2  20100416  IF1009                          3600.0  3647.8  3505.4    1730   
3  20100416  IF1012                          3618.8  3669.0  3551.2    2831   
6  20100419  IF1005                          3396.0  3398.0  3166.2  109733   

           成交金额     持仓量    持仓变化     今收盘     今结算     前结算    涨跌1    涨跌2  \
0  5.053881e+06  2702.0  2702.0  3415.6  3431.2  3399.0   16.6   32.2   
1  5.115764e+05   404.0   404.0  3441.6  3456.2  3399.0   42.6   57.2   
2  1.840498e+05   182.0   182.0  3512.0  3520.4  3399.0  113.0  121.4   
3  3.042605e+05   302.0   302.0  3557.0  3565.4  3399.0  158.0  166.4   
6  1.082064e+07  3954.0  1252.0  3197.4  3201.2  3431.2 -233.8 -230.0   

  隐含波动率(%) Delta  
0       --    --  
1       --    --  
2       --    --  
3       --